In [ ]:
import sqlite3
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,BaseMessage
from dotenv import load_dotenv
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph.message import add_messages  
import streamlit as st
from langsmith import traceable
import os
#tool nodes
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool

In [ ]:
load_dotenv()
llm= ChatOpenAI()

#Tools

In [ ]:
import requests
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool

# prebuilt tool
search_tool = DuckDuckGoSearchRun(region="us-en")

# custom calculator tool
@tool
def calculator(n1: float, n2: float, operation: str) -> dict:
    """
    Perform mathematical operations on two numbers.
    Supported operations: add, sub, mul, div
    """
    try:
        if operation == "add":
            result = n1 + n2
        elif operation == "sub":
            result = n1 - n2
        elif operation == "mul":
            result = n1 * n2
        elif operation == "div":
            if n2 == 0:
                return {"error": "division by zero not allowed"}
            result = n1 / n2
        else:
            return {"error": f'unsupported operation "{operation}"'}

        return {"n1": n1, "n2": n2, "operation": operation, "result": result}
    except Exception as e:
        return {"error": str(e)}

# custom stock price tool
@tool
def get_stock_price(symbol: str) -> dict:
    """
    Fetch the latest stock price for the given symbol (e.g., 'AAPL', 'TSLA')
    using the Alpha Vantage API.
    """
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=6OEI5ZVB2RS2AECF"
    r = requests.get(url)   
    return r.json()


In [ ]:
#create list of all tools
tools=[get_stock_price, calculator, search_tool]

llm_with_tools=llm.bind_tools(tools)

In [ ]:
#create state
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
def chat_node(state:State):
    messages=state['messages']
    response=llm_with_tools.invoke(messages)
    return {"messages":[response]}

tool_node= ToolNode(tools) #execute tool calls

In [ ]:
graph= StateGraph(State)
graph.add_node("Chat Node", chat_node)
graph.add_node("tools", tool_node)

#edge
graph.add_edge(START,"Chat Node")

graph.add_conditional_edges("Chat Node",tools_condition)
graph.add_edge("tools","Chat Node")

In [ ]:
graph.compile()

In [ ]:
chatbot=graph.compile()

In [ ]:
out=chatbot.invoke({"messages":[HumanMessage(content="Hello")]})
print(out["messages"][-1].content)

In [ ]:
out = chatbot.invoke({"messages": [HumanMessage(content="What is the stock price of Apple?")]})

print(out["messages"][-1].content)

In [ ]:
out = chatbot.invoke({"messages": [HumanMessage(content="What is the stock price of Apple?")]})

print(out["messages"][-1].content)

### Human In the Loop

In [1]:
from langchain_core.messages import AIMessage, AnyMessage, BaseMessage
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.graph import START, END, StateGraph
from langgraph.types import interrupt, Command  #for Human in the loop
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv


In [2]:
load_dotenv()
llm=ChatOpenAI()

In [3]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [5]:
def chat_node(state:State):
    decision=interrupt(
        {
            "type":"approvel",
            "reason":"Model is about to answer user question",
            "question": state["messages"][-1].content,
            "instructions":"Approve this question? yes/no"
        }
    )

    if decision["approved"]=='no':
        return{"messages":[AIMessage(content="Not approved")]}
    
    else:
        response=llm.invoke(state["messages"])
        return {"messages":[response]}



In [6]:
graph=StateGraph(State)

graph.add_node("chat node", chat_node)

graph.add_edge(START, "chat node")
graph.add_edge("chat node", END)

checkpointer=InMemorySaver()
workflow=graph.compile(checkpointer=checkpointer)

In [13]:
config={"configurable":{"thread_id":"123"}}

input_data = {
    "messages": [
        ("user", "Explain gradient descent in simple terms")
    ]
}


res = workflow.invoke(input_data, config=config)


In [14]:
res

{'messages': [HumanMessage(content='Explain gradient descent in simple terms', additional_kwargs={}, response_metadata={}, id='2805ad6e-22c3-44df-b02d-d44705f55922')],
 '__interrupt__': [Interrupt(value={'type': 'approvel', 'reason': 'Model is about to answer user question', 'question': 'Explain gradient descent in simple terms', 'instructions': 'Approve this question? yes/no'}, id='0f34fd0c1fa91f5461569d076a7e2e93')]}

In [15]:
message = res['__interrupt__'][0].value
message

{'type': 'approvel',
 'reason': 'Model is about to answer user question',
 'question': 'Explain gradient descent in simple terms',
 'instructions': 'Approve this question? yes/no'}

In [20]:
message_user_input = input(f" Backend Message --- {message}\n Approve this question? (yes/no): ")


In [21]:
final_res=workflow.invoke(Command(resume={"approved":message_user_input}),config=config)

In [22]:
print(final_res)

{'messages': [HumanMessage(content='Explain gradient descent in simple terms', additional_kwargs={}, response_metadata={}, id='2805ad6e-22c3-44df-b02d-d44705f55922'), AIMessage(content='Gradient descent is an optimization algorithm used to minimize a function by iteratively moving in the direction of steepest descent, which is the negative of the gradient of the function. In simpler terms, it is like trying to find the bottom of a valley by taking small steps downhill and adjusting your direction based on the slope of the terrain. By repeating this process, the algorithm can ultimately find the minimum value of the function.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 14, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'op

### SubGraphs

In [1]:
from typing import TypedDict
from langgraph.graph import START, END, StateGraph
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI

In [2]:
class State(TypedDict):
    input_text:str
    translate_text: str

In [3]:
subgraph_llm= ChatOpenAI()

In [4]:
def translate(state: State):
    prompt = f"""
You are a translator. Translate the following text into Punjabi.
Make sure it is accurate and point to point.

Text:
{state["input_text"]}
""".strip()

    translate_text = subgraph_llm.invoke(prompt).content
    return {"translate_text": translate_text}


In [5]:
subgraph_builder= StateGraph(State)
subgraph_builder.add_node("translate", translate)

subgraph_builder.add_edge(START, "translate")
subgraph_builder.add_edge("translate", END)

subgraph=subgraph_builder.compile()

In [6]:
#Parent class
class ParentState(TypedDict):
    question:str
    answer_gen:str
    ans_punj:str 

In [7]:
parent_llm= ChatOpenAI()

In [8]:
def generate_ans(state: ParentState):
    answer=parent_llm.invoke(f"You are a helful assistant. Answer clearly.\n\n Question: {state['question']}").content
    return {"answer_gen": answer}


In [12]:
def translator(state:ParentState):
    result=subgraph.invoke({'input_text': state["answer_gen"]})
    return {"ans_punj": result["translate_text"]}

In [13]:
parent_builder=StateGraph(ParentState)

parent_builder.add_node("answer", generate_ans)
parent_builder.add_node("translator",translator)

parent_builder.add_edge(START,"answer")
parent_builder.add_edge("answer", "translator")
parent_builder.add_edge("translator", END)

workflow=parent_builder.compile()


In [14]:
workflow.invoke({"question":"What is quantum physics?"})

{'question': 'What is quantum physics?',
 'answer_gen': 'Quantum physics is the branch of physics that studies the behavior of particles at a very small scale, such as atoms and subatomic particles. It involves principles such as quantization, wave-particle duality, and uncertainty. Quantum physics is different from classical physics in that it allows for the existence of superposition and entanglement among particles.',
 'ans_punj': 'ਕਵੰਟਮ ਭੌਤਿਕੀ ਉਹ ਭੌਤਿਕੀ ਸ਼ਾਖਾ ਹੈ ਜੋ ਖੁੱਲ੍ਹੇ ਦਾ ਭਾਵ ਦਾ ਅਧਿਯਯਨ ਕਰਦੀ ਹੈ, ਜਿਵੇਂ ਕਿ ਪਰਮਾਣੂ ਅਤੇ ਉਪਪਰਮਾਣੂ ਕਣਾਂ ਦੀ ਚਾਲਾਂ। ਇਸ ਵਿਚ ਇਕੁਈਕਰਣ, ਤਰੰਗ-ਪਰਮਾਣੂ ਦੋਗਲਤਾ, ਅਤੇ ਅਨਿਯਤਤਾ ਦੀਆਂ ਸੰਜਣਾਂ ਸ਼ਾਮਲ ਹੁੰਦੀਆਂ ਹਨ। ਕਵੰਟਮ ਭੌਤਿਕੀ ਨੂੰ ਕਲਾਸੀਕਲ ਭੌਤਿਕੀ ਤੋਂ ਇੱਕ ਹੋਰ ਹੈ ਕਿ ਇਸ ਨੂੰ ਕਣਾਂ ਵਿੱਚ ਅਕਾਰਾਂ ਵੱਲੋਂ ਅਤੇ ਉਪਸਰਨਾਂ ਦੇ ਹੋਣ ਦੀ ਇਜ਼ਾਜ਼ਤ ਦਿੰਦਾ ਹੈ।'}

### Shared state in Subgraph